# 01 - Basic Usage of Qurry

In Qurry, we use the instance of `Qurry` to store Quantum Circuit, experiment, build experiment, and excute them by local simulator or pending to the real quantum machine on `IBM`. In this chapter (or we say this jupyter file), we will introduce how to use Qurry to build a simple quantum circuit and measure their Renyi Entropy by local simulator.


If you are using Colab, you can run the following command in terminal to install Qurry.

```bash
pip install -i https://test.pypi.org/simple/ qurry
```

In [1]:
from IPython.display import display
from qurry import __version__

__version__

'0.7.2.dev1'

## 1.1 Setup Environment and Creat experiment executor

In [2]:
from qurry import EntropyMeasure
from qiskit import QuantumCircuit


- Import simulator

In [3]:
from qiskit import BasicAer

backend_qasm = BasicAer.get_backend('qasm_simulator')

/tmp/ipykernel_1312891/1158637932.py:1: DeprecationWarning: BasicAer is deprecated since Qiskit 0.46 and will be removed in Qiskit 1.0. The BasicAer (qiskit.providers.basicaer) module has been superseded by  qiskit.providers.basic_provider, and all its classes have been renamed to follow a new naming convention. More information and migration guidelines can be found in the 0.46 API docs for BasicAer.
  from qiskit import BasicAer


- Now, initialize our `EntropyMeasure`, we have two methods, `randomized` for Randomized Measure as default, and `hadamard` for Hadamard Test, these are our methods for measurement.

- In following tutorial, we will use `randomized` as our measurement method.

In [4]:
experiment_execution_01 = EntropyMeasure(method='hadamard')
print(f'| The first method is which is `Hadamard Test`".')
experiment_execution_02 = EntropyMeasure()
print(f'| The default method is which is `Randomized Measure` we will use.')

| The first method is which is `Hadamard Test`".
| The default method is which is `Randomized Measure` we will use.


---

## 1.2 Load quantum circuit

- We use `TrivialParamagnet` as our target to measure, and add it to our `Qurry` object, and import from `qurry.case` which there are some simple cases.

In [5]:
from qurry.recipe import TrivialParamagnet, GHZ

In [6]:
sample01 = TrivialParamagnet(8)
print("| trivial paramagnet in 8 qubits:")
print(sample01)
sample02 = GHZ(8)
print("| GHZ in 8 qubits:")
print(sample02)
sample03 = QuantumCircuit(8)
sample03.x(range(0,8,2))
print("| Custom circuit:")
print(sample03)

| trivial paramagnet in 8 qubits:
     ┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     ├───┤
q_2: ┤ H ├
     ├───┤
q_3: ┤ H ├
     ├───┤
q_4: ┤ H ├
     ├───┤
q_5: ┤ H ├
     ├───┤
q_6: ┤ H ├
     ├───┤
q_7: ┤ H ├
     └───┘
| GHZ in 8 qubits:
     ┌───┐                                   
q_0: ┤ H ├──■────────────────────────────────
     └───┘┌─┴─┐                              
q_1: ─────┤ X ├──■───────────────────────────
          └───┘┌─┴─┐                         
q_2: ──────────┤ X ├──■──────────────────────
               └───┘┌─┴─┐                    
q_3: ───────────────┤ X ├──■─────────────────
                    └───┘┌─┴─┐               
q_4: ────────────────────┤ X ├──■────────────
                         └───┘┌─┴─┐          
q_5: ─────────────────────────┤ X ├──■───────
                              └───┘┌─┴─┐     
q_6: ──────────────────────────────┤ X ├──■──
                                   └───┘┌─┴─┐
q_7: ───────────────────────────────────┤ X ├
                        

- After we prepare three quantum circuits, we can add them to our `Qurry` object.
The attribute `.waves` of `Qurry` object is a customized `dict`, we can check it by print it.

In [7]:
experiment_execution_02.add(
    wave=sample01,              # The circuit object
    key='TrivialParamagnet',    # The name of the circuit
    replace=False               # True for if you want to replace the circuit with the same name.
)                               # Default is False, so if you add the same name when False, 
                                # it will be added with serial number.
experiment_execution_02.add(sample02, 'GHZ')
experiment_execution_02.add(sample03)
print(experiment_execution_02.waves)

<WaveContainer={
    TrivialParamagnet: ...
    GHZ: ...
    2: ...
} with 3 waves load, a customized dictionary>


If you do not give the name of circuit, it will be named by series number.

Now, waves are loading, time to excute.

## 1.3 Excute Circuits

- We can use `.measure()` to excute the specific circuit in `EntropyMeasure` object, and it will return hash id of this experiment and save a `Experiment` object in `EntropyMeasure`.
- For `EntropyMeasure`, it will require the following parameters:
  - wave: the name of circuit in `Qurry.waves` object
  - times: the number of random measurement
  - shots: the number of shots for each measurement. Default is 1024

In [8]:
exp1 = experiment_execution_02.measure(
    wave='TrivialParamagnet',
    times=100,
    shots=1024,
    backend=backend_qasm
)
exp1 

'972e7f90-7e75-4e51-b698-6d83454e543e'

- You can specify the backend by add a parameter `backend` in `measure` method, if you do not specify, it will use simulator as the default backend. Moreover, qiskit has multiple simulators, they are `QasmSimulatorPy` from `qiskit`, `AerSimulator` from `qiskit-aer`, and `AerSimulator` from `qiskit-aer-gpu`. Qurry will check the availability of these simulators and use the first available one as the default backend in following order:
  1. `AerSimulator` from `qiskit-aer-gpu`
  2. `AerSimulator` from `qiskit-aer`
  3. `QasmSimulatorPy` from `qiskit`

In [9]:
exp2 = experiment_execution_02.measure(
    wave='TrivialParamagnet',
    times=100,
    shots=1024,
)
exp2

'25c287a6-738b-4ef0-9903-7090c31bd19c'

- And we can check the attribute `.exps` of `EntropyMeasure` object, it will return the hash id of all experiments.

In [10]:
print(experiment_execution_02.exps)

<ExperimentContainer={
    972e7f90-7e75-4e51-b698-6d83454e543e: ...
    25c287a6-738b-4ef0-9903-7090c31bd19c: ...
} with 2 experiments load, a customized dictionary>


- The following is the other informatiom you can access.

In [11]:
experiment_execution_02.exps[exp1]

<qurrentRandomized.Experiment with exp_id=972e7f90-7e75-4e51-b698-6d83454e543e, EntropyRandomizedArguments(exp_name='w=TrivialParamagnet.with100random.qurrent_haar', times=100, measure=(0, 8), unitary_loc=(0, 8), workers_num=16), Commonparams(exp_id='972e7f90-7e75-4e51-b698-6d83454e543e', wave_key='TrivialParamagnet', shots=1024, backend=<QasmSimulatorPy('qasm_simulator')>, run_args={}, transpile_args={}, tags=(), default_analysis=[], save_location=PosixPath('.'), filename='', files={}, serial=None, summoner_id=None, summoner_name=None, datetimes={'build': '2024-04-08 01:23:09', 'run': '2024-04-08 01:23:10'}), 0 unused arguments, 6 preparing dates, 2 experiment result datasets, and 0 analysis>

In [12]:
print(experiment_execution_02.exps[exp1].statesheet())

 # qurrentRandomized.Experiment with exp_id=972e7f90-7e75-4e51-b698-6d83454e543e
 - arguments
   - exp_name ---------------- w=TrivialParamagnet.with100random.qurrent_haar
   - times ------------------- 100
   - measure ----------------- (0, 8)
   - unitary_loc ------------- (0, 8)
   - workers_num ------------- 16
 - commonparams
   - exp_id ------------------ 972e7f90-7e75-4e51-b698-6d83454e543e   # This is ID is generated by Qurry which is different from 'job_id' for pending.
   - wave_key ---------------- TrivialParamagnet
   - shots ------------------- 1024
   - backend ----------------- qasm_simulator
   - run_args ---------------- {}
   - transpile_args ---------- {}
   - tags -------------------- ()
   - default_analysis -------- []
   - save_location ----------- .
   - filename ---------------- 
   - files ------------------- {}
   - serial ------------------ None
   - summoner_id ------------- None
   - summoner_name ----------- None
   - datetimes --------------- {'build': '

- Then, there is a experiment completed, and we can calculate the Renyi Entropy by `analyze`.

---

## 1.4 Result

- We want to know the Renyi Entropy of half system. Then we can use `.analyze()` to calculate it and it will store a `Analysis` object in `Experiment`.
- The following is the parameter `analyze` of `EntropyMeasure` required.
  - degree: The size or range of subsystem, if it is a number, it will be the size of subsystem, if it is a tuple, it will be the range of subsystem.

### Bit select hint

A bit string mapping to the index of qubit is `'76543210'` for 8 qubits example here.

| the input | bitStringRange | String Slice | Actual Qubit Included |
| --- | --- | --- | --- |
| `2` | `(6, 8)` | `'10'` | 01 |
| `4` | `(4, 8)` | `'3210'` | 0123 |
| `6` | `(2, 8)` | `'543210'` | 012345 |
| `(2, 4)` | `(2, 4)` | `'54'` | 45 |
| `(4, 6)` | `(4, 6)` | `'32'` | 23 |
| `(2, 6)` | `(2, 6)` | `'5432'` | 2345 |
| `(-2, 2)` | `(-2, 2)` | `'1076'` | 6701 |
| `(-4, 2)` | `(-4, 2)` | `'321076'` | 670123 |
| `(-2, 4)` | `(-2, 4)` | `'107654'` | 456701 |
| `3` | `(5, 8)` | `'210'` | 012 |
| `(5, 7)` | `(5, 7)` | `'21'` | 12 |
| `(4, 7)` | `(4, 7)` | `'321'` | 123 |


In [13]:
exp1_report1 = experiment_execution_02.exps[exp1].analyze(
    degree=(3, 4)
)
exp1_report2 = experiment_execution_02.exps[exp1].analyze(
    degree=1
)

|  - 00:00 < ?

|  - 00:00 < ?

- The `analyze` will return the result of this analysis.

In [14]:
print(exp1_report1)
print(exp1_report1.header)

<qurrentRandomized.Analysis with serial=0, AnalysisInput(degree=(3, 4), shots=1024, unitary_loc=(0, 8)), AnalysisContent(purity=0.9235139083862305, entropy=0.11479440611197787, and others), 0 unused arguments>
AnalysisHeader(serial=0, datetime='2024-04-08 01:23:11', summoner=None, log={})


- It can also be found in the attribute `.reports` of `Experiment` object.

In [15]:
print(experiment_execution_02.exps[exp1].reports)
print(experiment_execution_02.exps[exp1].reports[0])
print(experiment_execution_02.exps[exp1].reports[0].header)

<AnalysisContainer={
    0: ...
    1: ...
} with 2 analysis load, a customized dictionary>
<qurrentRandomized.Analysis with serial=0, AnalysisInput(degree=(3, 4), shots=1024, unitary_loc=(0, 8)), AnalysisContent(purity=0.9235139083862305, entropy=0.11479440611197787, and others), 0 unused arguments>
AnalysisHeader(serial=0, datetime='2024-04-08 01:23:11', summoner=None, log={})


- We can use `.export()` to get the result of this analysis.

In [16]:
main, side_prodct = experiment_execution_02.exps[exp1].reports[0].export()

- And there is it

In [17]:
main

{'purity': 0.9235139083862305,
 'entropy': 0.11479440611197787,
 'puritySD': 0.4376245218163771,
 'entropySD': 0.6836483150526039,
 'bitStringRange': [3, 4],
 'allSystemSource': 'independent',
 'purityAllSys': 1.1075023651123046,
 'entropyAllSys': -0.14730977971225714,
 'puritySDAllSys': 1.38595359780548,
 'entropySDAllSys': 1.8054213204803637,
 'bitsStringRangeAllSys': [0, 8],
 'errorRate': -0.05257966112532599,
 'mitigatedPurity': 0.8822590088935455,
 'mitigatedEntropy': 0.18072583820802568,
 'num_qubits': 8,
 'measure': ['measure range:', [0, 8]],
 'measureActually': [0, 8],
 'measureActuallyAllSys': [0, 8],
 'countsNum': 100,
 'takingTime': 0.000714141,
 'takingTimeAllSys': 0.003762497,
 'input': {'degree': [3, 4], 'shots': 1024, 'unitary_loc': [0, 8]},
 'header': {'serial': 0,
  'datetime': '2024-04-08 01:23:11',
  'summoner': None,
  'log': {}}}

- Also, `sideProdct` will record the data of each circuit, and you can use it to do more analysis.

In [18]:
side_prodct

{'purityCells': {97: 1.192962646484375,
  41: 1.018423080444336,
  34: 1.9534912109375,
  42: 1.2170639038085938,
  89: 1.4431991577148438,
  48: 1.543294906616211,
  21: 1.753265380859375,
  22: 0.6287460327148438,
  72: 0.5837764739990234,
  31: 0.7620468139648438,
  49: 0.5000057220458984,
  33: 1.067770004272461,
  18: 1.633108139038086,
  68: 1.258148193359375,
  75: 0.702239990234375,
  98: 0.9945907592773438,
  78: 0.797454833984375,
  45: 1.161468505859375,
  56: 0.6446590423583984,
  5: 0.5837764739990234,
  30: 1.6128292083740234,
  39: 1.791036605834961,
  28: 1.046346664428711,
  53: 0.5166854858398438,
  60: 0.6732406616210938,
  64: 0.612152099609375,
  67: 1.361419677734375,
  88: 0.6042842864990234,
  91: 0.9845485687255859,
  62: 0.5505599975585938,
  4: 0.5473842620849609,
  54: 1.5777816772460938,
  80: 0.5087032318115234,
  90: 1.225189208984375,
  96: 0.5030269622802734,
  84: 0.511077880859375,
  99: 0.6673183441162109,
  44: 0.8838405609130859,
  74: 1.9650497436

In [19]:
experiment_execution_02.exps[exp1].reports[0].statesheet()

 # qurrentRandomized.Analysis with serial=0
 - header
   - serial ----------------------- 0
   - datetime --------------------- 2024-04-08 01:23:11
   - summoner --------------------- None
   - log -------------------------- {}
 - input
   - degree ----------------------- (3, 4)
   - shots ------------------------ 1024
   - unitary_loc ------------------ (0, 8)
 - outfields -------------------- 0 ........  # Number of unused arguments.
 - content
   - purity ----------------------- 0.9235139083862305
   - entropy ---------------------- 0.11479440611197787
   - puritySD --------------------- 0.4376245218163771
   - entropySD -------------------- 0.6836483150526039
   - purityCells ------------------ {97: 1.192962646484375, 41: 1.018423080444336, 34: 1.9534912109375, 42: 1.2170639038085938, 89: 1.4431991577148438, 48: 1.543294906616211, 21: 1.753265380859375, 22: 0.6287460327148438, 72: 0.5837764739990234, 31: 0.7620468139648438, 49: 0.5000057220458984, 33: 1.067770004272461, 18: 1.63310

---

## Post-Process Availablities and Version Info

We currently do not support Qsikit 1.0 for we are working on it, and we will support it in the future.

In [20]:
from qurry.process import AVAIBILITY_STATESHEET
from qurry.tools.qiskit_version import QISKIT_VERSION_STATESHEET

print(AVAIBILITY_STATESHEET)
print(QISKIT_VERSION_STATESHEET)

 | Qurry version: 0.7.2.dev1
--------------------------------------------------------
 ### Qurry Post-Processing
   - Backend Availability ......... Python Cython Rust  
 - randomized_measure
   - entangled_core ............... True   True   True  
   - purity_cell .................. True   True   True  
   - wavefunction_overlap ......... True   True   None  
   - echo_cell .................... True   True   None  
 - utils
   - randomized ................... True   True   True  
   - construct .................... True   None   True  
--------------------------------------------------------
   + True ..... Working normally.
   + False .... Exception occurred.
   + None ..... Not supported yet.
--------------------------------------------------------

 | Qurry version: 0.7.2.dev1
--------------------------------------------
 ### Qiskit version
 - main
   - qiskit-aer ................... 0.11.2
   - qiskit-aer-gpu ............... 0.11.2
   - qiskit-ibm-provider .......... 0.7.3
   - qi